In [1]:
import pickle
import copy
from scipy import stats
import itertools
from operator import itemgetter
import pandas as pd
import numpy as np
import os
import glob
import mdtraj as md
import tqdm

In [3]:
# Using this cose you should use a MD that starts with a conformation and has a change in the midle of the MD and then
# adopts another conformation. So, cut your MD file to be like that. I normaly clusterize my simulation and save smaller
# MD's for the 2 conformations/clusters I want to use (I keep the resulting MD's with the same ammount of frames because "Statistics").
# Then I load all them in the same variable as follows:

MD = md.load(['CLUSTERS/PCA_5-6/THM_ATP_1.dcd','CLUSTERS/PCA_5-6/GCV_ATP_3.dcd'],top='/home/janet/Desktop/HSVTK/TOPS/MD_1E2K_GCV_ATP.pdb')
MD = MD.atom_slice(MD.top.select('protein'))

#Remember to know in what frame of this "MD" you have the conformational change. eg: MD1(2000 frames) + MD2(2500 frames): Change = 2000
# This is important because you will introduce this value later as the variable "key_frame"

In [ ]:
# Now let's compute all the contacts for all the residues. Only residues that are at 5A at some point of the MD will be
# computed. This is computed for all the atoms in the residue, not only CA. This value is enough, but if you want to change
# it you can do it in the 15 line of the next cel.

In [4]:
#nres = numero de residus del sistema
#contacts will include the information the residues involved in the computed distance (shape= residue1,close_residues)
#distances will include distances computed in arrays(shape = residue1,frames,close_residues)
#neigh will include all atoms close to the ones in the selected residue (all in the tqdm loop) and for ll the simulation frames
#merged_neig contains all atoms that are some point in the simulations are close to the selected residue
#residues list contains residues that are affected selecting all atoms in merged_neigh
#with residues, all contacts between close residues are computed and saved as contacts and distances using 'closest-heavy' scheme

top = MD.top
nres = [res.index for res in top.residues]
contacts = []
distances = []
for i in tqdm.tqdm(list(range(len(nres)))):
    atoms_in_residue = [atom.index for atom in top.residue(i).atoms]
    neigh = md.compute_neighbors(MD,0.5,atoms_in_residue)
    merged_neigh = list(set([b for a in neigh for b in a]))
    merged_neigh.sort(key=int)
    residues = list(set([top.atom(atn).residue.index for atn in merged_neigh]))
    residues.sort(key=int)
    new_res = []
    for res in residues:
        new_res.append(res)
    residues = new_res
    distances_i, contacts_i = md.compute_contacts(MD,contacts=list(zip([i]*len(residues),residues)), scheme='closest-heavy')
    contacts.append(contacts_i) ; distances.append(distances_i)
    del merged_neigh
    del neigh
    

100%|██████████| 660/660 [28:00<00:00,  2.55s/it]


In [5]:
#I reeeeeeeeeeally recomend you to save the data you generated in a pickLe

contacts_file = open('contacts_GCV_ATP_1-3.p', 'wb')
distances_file = open('distances_GCV_ATP_1-3.p', 'wb')
pickle.dump(contacts, contacts_file)
pickle.dump(distances, distances_file)
contacts_file.close()
distances_file.close()

In [ ]:
# singleKS function will compute the kolmogorov-smirnov test of one contact between the 2 conformations and output the data (This is only for visualization porpuses)
# allKS will compute the kolmogorov-smirnov test for all the contacts between the 2 conformations (This will take long, probably)
# respairs will compute the residues pairs with higher KS values (positive or negative).
# atom_pairs will compute the closest atoms between the residues. This are normaly the ones that make the contact, but maybe not. Please check that the conversion Residue-Atom is ok
# pymol_script will create you a script for visualizating this data on a session with the topology open. Please chech that the Residues shown are ok. Numbering between this libraries and programs are different. Also, maybe in dimers/tetramers, ... this can get messy


In [6]:
#Remember the 'key_frame' variable I told you? Time to introduce the value
key_frame = 2000

#def singleKS(data1, data2):
#    KS = stats.ks_2samp(data1, data2)
#    frequencies1, edges1 = np.histogram(data1, 20)
#    frequencies2, edges2 = np.histogram(data2, 20)
#    hist1 = hv.Histogram((edges1, frequencies1), labels='Data1')
#    hist2 = hv.Histogram((edges2, frequencies2), labels='Data2')
#    return (hist1 * hist2).opts(width=600, title=f'K-M value: {KS[0]:.3f}          p-Value: {KS[1]:.3f}')

def allKS(distances, contacts, top, key_frame):
    KS_outs = copy.deepcopy(contacts)
    for i in tqdm.tqdm(range(len(contacts))):
        KS_outs[i] = KS_outs[i].astype('float64')
        for j in range(len(contacts[i])):
            out = stats.ks_2samp(distances[i][:,j][:key_frame], distances[i][:,j][key_frame:])
            KS_outs[i][j] = out[0], out[1]
    return KS_outs

def respairs(KS, contacts, lim=10):
    pairs = []
    KSS = sorted([KS[r1][r2][0] for r1 in range(len(KS)) for r2 in range(len(KS[r1]))])[::-1][:lim]
    for r1 in range(len(KS)):
        for r2 in range(len(KS[r1])):
            if KS[r1][r2][0] in KSS:
                pairs.append([contacts[r1][r2][0], contacts[r1][r2][1]])
    return pairs

def atom_pairs(pairs,MD):
    MD_top = MD[0]
    top = MD.top 
    heavy_pairs = []
    for i,j in pairs:
        atoms1 = [atom.index for atom in top.residue(i).atoms if atom.element.name != 'hydrogen']
        atoms2 = [atom.index for atom in top.residue(j).atoms if atom.element.name != 'hydrogen']
        atom_pairs = list(itertools.product(atoms1, atoms2))
        heavy_distances = md.compute_distances(MD_top,atom_pairs)
        heavy_pairs.append(atom_pairs[np.argmin(heavy_distances)])
    return heavy_pairs

def pymol_script(pairs, top, name=''):
    f = open(f'pymol_distances_{name}', 'w+')
    f.write(f'hide all\n')
    f.write(f'show cartoon\n')
    for pair in pairs:
        f.write(f'distance {top.atom(pair[0]+1).residue}-{top.atom(pair[1]+1).residue}, id {pair[0]+1}, id {pair[1]+1}\n')
        f.write(f'show sticks, resi {top.atom(pair[0]).residue.index+1}\n')
        f.write(f'show sticks, resi {top.atom(pair[1]).residue.index+1}\n')
    f.write('hide (hydro)')
    f.close()

In [ ]:
# let's do it 1 by 1

In [7]:
# Compute all Kolmogorow-Smirnov tests
KS = allKS(distances,contacts,top,key_frame)

100%|██████████| 660/660 [00:19<00:00, 33.59it/s]


In [8]:
# Show the 20 residue contacts that change the most. Negative values for KS (-1) mean anti-correlation 
lim=20
respairs(KS, contacts, lim)

[[16, 37],
 [16, 39],
 [16, 658],
 [20, 116],
 [20, 117],
 [37, 16],
 [37, 117],
 [39, 16],
 [116, 20],
 [117, 20],
 [117, 37],
 [219, 224],
 [220, 224],
 [220, 225],
 [224, 219],
 [224, 220],
 [225, 220],
 [550, 554],
 [551, 554],
 [554, 550],
 [554, 551],
 [658, 16]]

In [9]:
# Please check thet the atom pairs are well computed.
pairs = atom_pairs(respairs(KS, contacts, lim),MD)
pairs

[(262, 587),
 (262, 616),
 (262, 10014),
 (316, 1720),
 (316, 1732),
 (587, 262),
 (587, 1745),
 (616, 262),
 (1720, 316),
 (1732, 316),
 (1745, 587),
 (3352, 3419),
 (3362, 3419),
 (3362, 3424),
 (3419, 3352),
 (3419, 3362),
 (3424, 3362),
 (8376, 8433),
 (8399, 8431),
 (8433, 8376),
 (8431, 8399),
 (10014, 262)]

In [11]:
# Visualize the data. Enjoy!
pymol_script(atom_pairs(respairs(KS, contacts,lim),MD), top, 'GCV_ATP_1.pml')